In [3]:
from llama_cpp import Llama
# プロンプトを記入
prompt = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write a story about llamas.Please answer in Japanese.[/INST]"""
# ダウンロードしたModelをセット.
llm = Llama(model_path=r"llama.cpp\models\rinna-youri-7b-chat-q4_K_M.gguf", n_gpu_layers=20)
# 生成実行
output = llm(
    prompt,max_tokens=500,stop=["System:", "User:", "Assistant:"],echo=True,
)
print(output)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


{'id': 'cmpl-8c5e118d-9e7c-4635-8616-840241ad3418', 'object': 'text_completion', 'created': 1700977566, 'model': 'llama.cpp\\models\\rinna-youri-7b-chat-q4_K_M.gguf', 'choices': [{'text': "[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.If you don't know the answer to a question, please don't share false information.\n<</SYS>>\nWrite a story about llamas.Please answer in Japanese.[/INST]", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 145, 'completion_tokens': 0, 'total_tokens': 145}}


In [16]:
import logging
import sys

from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores.faiss import FAISS
from typing import Any

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# LangChain用に句読点で分割してくれるTextSplitterを作ってみた
# https://www.sato-susumu.com/entry/2023/04/30/131338
class JapaneseCharacterTextSplitter(RecursiveCharacterTextSplitter):
    def __init__(self, **kwargs: Any):
        separators = ["\n\n", "\n", "。"]  #, "、", " ", "、","　"]
        super().__init__(separators=separators, **kwargs)

# ドキュメントの読み込み
# with open("bocchi_en.txt") as f:
with open("frielen_ja.txt") as f:
    test_all = f.read()

# チャンクの分割
# chunk_size が長すぎると時間がかかる、短すぎると（5以下）間違った回答になりやすい.40くらいがいいかも。
japanese_spliter = JapaneseCharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=0,
)
texts = japanese_spliter.split_text(test_all)

'''
# spacyはそのままだと日本語は分割できない 
text_splitter = SpacyTextSplitter(
    chunk_size=1000  # チャンクのトークン数
)
texts = text_splitter.split_text(test_all)
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=300,  # チャンクの最大文字数
#     chunk_overlap=20,  # オーバーラップの最大文字数
# )
# texts = text_splitter.split_text(test_all)
'''

# チャンクの確認
print(len(texts))
for text in texts:
    print(text[:10].replace("\n", "\\n"), ":", len(text))


# インデックスの作成
index = FAISS.from_texts(
    texts=texts,
    # embedding=HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large"),
    # embedding=HuggingFaceEmbeddings(model_name="sonoisa/sentence-bert-base-ja-mean-tokens-v2"),
    embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite", encode_kwargs={"normalize_embeddings":True })
)
index.save_local("storage")

# インデックスの読み込み
# index = FAISS.load_local(
#    "storage", HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
# )

# LLMの準備
# llm = OpenAI(temperature=0, verbose=True)
# LlamaCpp(model_path=model, verbose=True, n_ctx=2048)
llm = LlamaCpp(
    model_path="llama.cpp/models/rinna-youri-7b-chat-q4_K_M.gguf",
    # model_path=r"llama.cpp/cpp\models\llama-2-7b-chat.Q4_K_M.gguf",
    input={
        "max_tokens": 32,
        "stop": ["System:", "User:", "Assistant:", "\n"],
    },
    verbose=True,
    temperature=1,
    n_ctx=2048
)

# 質問応答チェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.as_retriever(search_kwargs={"k": 4}),
    verbose=True,
)

# 質問応答チェーンの実行
print("A1:", qa_chain.run("フリーレンはどういう人ですか?"))
# print("A2:", qa_chain.run("ハイターはどういう人ですか?"))
# print("A1:", qa_chain.run("What kind of person is Hitori Goto?"))
# print("A2:", qa_chain.run("What instrument is Hitori Goto good at?"))
# print("A3:", qa_chain.run("What did Hitori Goto do at the school festival?"))

26
魔王を倒して王都に凱 : 36
魔法使いフリーレンら : 21
10年間もの旅路を終 : 23
1000年は軽く生き : 31
その旅はきわめて短い : 21
50年に一度降るとい : 30
次回もそれを見る約束 : 32
すっかり年老いたヒン : 24
ハイターやアイゼンと : 26
。まもなくヒンメルは : 16
彼の葬儀でフリーレン : 28
知ろうともしなかった : 38
人間を知るためと魔法 : 31
フリーレンは老い先短 : 19
。ハイターは魔導書の : 39
。その4年後、 : 7
フリーレンは魔導書の : 16
フェルンは1人前の魔 : 16
。ハイターの最期を看 : 28
さらにその後、\nアイ : 26
彼の協力によりフリー : 35
死者の魂と対話できる : 38
。かつての魔王城の所 : 26
ヒンメルとの再会を目 : 19
。そしてアイゼンの弟 : 29
フリーレン達の旅は続 : 12
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: oshizo/sbert-jsnli-luke-japanese-base-lite
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it]
c:\Users\onowa\miniconda3\envs\vir_env\lib\site-packages\langchain\utils\utils.py:159: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  warnings.warn(
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 




> Entering new RetrievalQA chain...


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.64it/s]



> Finished chain.
A1:   彼女は友達を訪ねる旅を続けており、同じくらい年老いたヒンメルに会いました。


In [19]:
# https://qiita.com/shimajiroxyz/items/91556e85254d5fff7f48
# 【langchain】ベクトルストアの保存と読込方法を整理する(save_local vs pickle)
# インデックスの読み込み
index = FAISS.load_local(
    "storage", HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
)
print(index)
# print(index.similarity_search_with_score("こんにちは"))
# print("A1:", qa_chain.run("フリーレンはどういう人ですか?"))
# print("A2:", qa_chain.run("ハイターはどういう人ですか?"))

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [29]:
print(index)
# print(index.similarity_search_with_score("フリーレン", k=5))

In [21]:
# A2 = "ハイターはどういう人ですか?"
A2 = "ヒンメルは何をしましたか？"
print(A2, ":", qa_chain.run(A2))




> Entering new RetrievalQA chain...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.34it/s]
Llama.generate: prefix-match hit



> Finished chain.
ヒンメルは何をしましたか？ :   旅をする


In [ ]:
# https://zenn.dev/octu0/scraps/4f26b8e6a170c5
from langchain.llms import LlamaCpp

model_path = r".\models\rinna-youri-7b-chat-q4_K_M.gguf"

stop = [
    '\nシステム: ',
    '\n\tシステム: ',
    '</s>'
]
llm = LlamaCpp(
    model_path=model_path,
    n_ctx=2048,
    max_tokens=1024,
    seed=1,
    stop=stop,
    f16_kv=True,
    verbose=True,
)
prompt="""ユーザー: 日本のおすすめの観光地を教えてください。<NL>
システム: どの地域の観光地が知りたいですか？<NL>
ユーザー: 渋谷の観光地を教えてください。<NL>
システム:"""

r = llm(prompt)
print(r)